# Introducción

En la siguiente notebook se presentará la consigna a seguir para el tercer práctico del proyecto, correspondiente a la materia Introducción al Aprendizaje Automático. El objetivo consiste en explorar la aplicación de diferentes métodos de aprendizaje supervisado aprendidos en el curso, a través de experimentos reproducibles, y evaluando a su vez la conveniencia de uno u otro, así como la selección de diferentes hiperparámetros a partir del cálculo de las métricas pertinentes.

En este caso, enfrentamos un problema de clasificación binario de posicionamiento respecto de un tópico. Para este práctico vamos a utilizar únicamente los datos etiquetados, que ya vienen divididos en train y test. Buscamos analizar distintos problemas que puedan surgir como el desbalanceo de clases


## Organización

El trabajo va a estar organizado en dos grandes secciones: preprocesamiento y aplicación de los clasificadores.

#### Preprocesamiento
En la parte de preprocesamiento lo que vamos a hacer va a ser:

1 - Obtener el dataset

2 - Tokenizar

3 - Aplicar alguna curación

4 - Balanceo de clases

5 - Representar el texto como vector: CountVectorizer

6 - Optativo: se puede representar el texto de otras maneras? Embeddings!

#### Clasificadores

1 - Perceptron

2 - K-NN

3 - Regresión Logística

4 - Evaluación de los clasificadores

5 - Optimización de Hiperparámetros

Esto para los tres datasets CON y SIN balanceo de clases


In [108]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report
from sklearn.model_selection import GridSearchCV

# Preprocesamiento

### Cargamos los datos

In [65]:
train = pd.read_csv("train.csv", sep=',', encoding="latin1").fillna(method="ffill")
test = pd.read_csv("test.csv", sep=',', encoding="latin1").fillna(method="ffill")

In [66]:
abortion_train = train[train["Target"] == "Legalization of Abortion"]
abortion_test = test[test["Target"] == "Legalization of Abortion"]

climate_train = train[train["Target"] == "Climate Change is a Real Concern"]
climate_test = test[test["Target"] == "Climate Change is a Real Concern"]

feminism_train = train[train["Target"] == "Feminist Movement"]
feminism_test = test[test["Target"] == "Feminist Movement"]

In [67]:
climate_train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
613,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other
614,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
615,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
616,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
617,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,3. The tweet is not explicitly expressing opi...,other


Sólo vamos a usar el tweet y el stance. Como encima ya tenemos dividido el corpus según el target, vamos a eliminar todas las columnas excepto tweet y stance

In [68]:
abortion_train.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)
abortion_test.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)

climate_train.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)
climate_test.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)

feminism_train.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)
feminism_test.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)

/home/damifur/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### Tokenizamos

En principio como mínimo para realizar algún tipo de preprocesamiento y luego transformar nuestros datos en algo que pueda ser tomado como input por los clasificadores que vamos a probar necesitamos dividir nuestro tweet en formato string en una lista de palabras. La división requiere tomar decisiones sobre cómo tratar anomalías. En especial en twitter donde abundan las abreviaciones, errores ortográficos, puntuaciones raras, emojis, lo que se le ocurra al usuario.

Hay muchas formas distintas de tokenizar y hay clasificadores que vienen con tokenizadores especiales incorporados al punto tal de que no pueden funcionar con otra tokenización (fastText y BERT por ejemplo separan la raíz de las parabras de sus prefijos y sufijos para poder relacionar palabras similares, como asociar todas las conjugaciones de un verbo a una misma raíz).

Nosotros vamos a usar uno bien simple que tiene pocas funciones pero tiene algunas funciones pensadas especialmente para redes sociales, como por ejemplo detectar emojis o separar una palabra de sus signos de puntuación o asociar muchos signos de puntuación iguales y seguidos como si fueran uno solo (por ejemplo, !!!!!).

https://www.nltk.org/api/nltk.tokenize.html

Hay tres parámetros que pueden explorar leyendo la documentación (los que están escritos). Prueben ver qué pasa cuando cambian cada uno

In [69]:
tokenizer = TweetTokenizer(preserve_case=True, reduce_len=True, strip_handles=False)

In [70]:
tokenizer.tokenize(abortion_train["Tweet"].iloc[1])

['@tooprettyclub',
 'Are',
 'you',
 'OK',
 'with',
 '#GOP',
 'males',
 'telling',
 'you',
 'what',
 'you',
 'can',
 'and',
 "can't",
 'do',
 'with',
 'your',
 'own',
 'body',
 '?']

### Preprocesamiento

El preprocesamiento para twitter requiere tomar varias decisiones. Recomiendo que vean un poco el dataset y piensen con qué palabras quieren trabajar y cuales quieren remover. Les dejo el esqueleto de una función de preprocesamiento que sólo tokeniza pero que puede tomar dos parámetros optativos para remover hashtags y números.

#### Ejercicio 1

Agregarle a la función de preprocesamiento que borre las urls (palabras que empiecen con http). Agregarle código para que agregue o saque texto de acuerdo con al menos un criterio propuesto por ustedes (menciones a los usuarios, caritas/emojis, puntuación).

In [71]:
def preprocesar(text, keep_hashtags=True, remove_numbers=False):
    toks = tokenizer.tokenize(text)

    ret = []
    for tok in toks:
        if tok[0] == "#" and not keep_hashtags:
            continue
            
        if tok.isnumeric() and remove_numbers:
            continue
        ret.append(tok)
    return " ".join(ret)

In [72]:
# Hacer esto para todos los datasets, train y test de los 3 tópicos
abortion_train["Tweet"] = abortion_train["Tweet"].apply(lambda x: preprocesar(x))

/home/damifur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [73]:
abortion_train.head()

,Tweet,Stance
50,Just laid down the law on abortion in my bioet...,AGAINST
51,@tooprettyclub Are you OK with #GOP males tell...,FAVOR
52,"If you don't want your kid , put it up for ado...",AGAINST
53,"@RedAlert - there should be a "" stigma "" to bu...",AGAINST
54,But isn't that the problem then . Not enough f...,NONE


### Balanceo de clases

En el práctico anterior ya analizaron la distribución de las clases según cada target (tópico) del dataset. Queremos explorar la posibilidad de hacer un balanceo de clases. Para eso, analizamos tópico por tópico si esto es posible y las dificultades que tiene.

Para el dataset de feminismo, tenemos dos versiones de train, una con correcciones y otra sin. En la versión con correcciones cambia el balanceo de las clases a algo más equitativo que en su version original. Por este motivo, vamos a descartar hacer un balanceo de clases en este dataset.

Para el dataset de cambio climático tenemos un desbalanceo tan grande que, por ejemplo, sólo tenemos 11 ejemplos de la clase Against (el 6,5% del corpus). El problema que tiene una distribución tan desigual es que resulta dificil aplicar técnicas como el subsampling porque nos quedaríamos con 33 tweets de entrenamiento, o el oversampling porque para que las clases queden parejas, deberíamos repetir un mismo tweet muchas veces.

Por lo tanto, nos queda el corpus de aborto:

#### Ejercicio 2

Hacer subsampling del corpus de aborto y guardarlo como un nuevo dataset. A partir de ahora, todos los experimentos que corran deberán correrlos además de para los tres corpus respectivos a cada tópico, también para este nuevo corpus de aborto con sus clases balanceadas. Luego vamos a comparar los resultados obtenidos con y sin balanceo de clases

In [ ]:
abortion_train_balanced = # TODO

### Representación como vector

Los algoritmos de Machine Learning trabajan con espacios vectoriales. Entonces siempre que trabajemos con Procesamiento de Lenguaje Natural, como es nuestro caso, se plantea la cuestión de cómo representar texto con números. Hay muchas maneras de hacer esto y es un campo que sigue evolucionando con el tiempo. Una opción muy básica es asignarle a cada palabra que aparece en nuestro dataset un número según el orden en el que aparecen. Luego, una oración es un vector de índices de esas palabras. Pero el problema que tiene esto es que los algoritmos de Machine Learning también requieren que los vectores tengan una longitud fija, con lo cual hay que recortar la oración o agregarle ceros al final. Por eso un enfoque clásico para representar texto es el Bag Of Words: un vector de bits del tamaño de todo nuestro vocabulario que tiene un uno si la palabra está en la oración y un 0 si no está.

https://es.wikipedia.org/wiki/Modelo_bolsa_de_palabras
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

En particular, vamos a usar la libreria CountVectorizer que implementa el Bag Of Words de manera esparsa (eficiente) y le agrega varios features que van a sernos muy útiles:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

En particular, vamos a usar los parámetros min_df y max_df que se corresponden con min y max document frequency. Ámbos toman valores entre 0 y 1 y estipulan el rango de frecuencia de aparición de una palabra dentro de un documento que vamos a aceptar. Es decir, si min_df es 0.005, todas las palabras que representen menos de un 0,5% de las palabras totales serán descartadas. Por el otro lado, si max_df es 0.35, todas las palabras que representen más de un 35% del total de palabras serán descartadas. Nos interesa descartar las palabras con demasiada frecuencia porque probablemente no tengan valor en términos de la entropía que aportan (es decir, no aportan información: pueden ser conectores, artículos, etc.) y las que tienen muy poca frecuencia porque pueden ser outliers, palabras demasiado específicas que no aportan a la tarea que queremos desarrollar.

Las Bag Of Words, sin embargo, tienen un problema importante: no preservan el contexto y la relación semántica de las palabras entre sí. Este problema dio lugar a otros enfoques como los embeddings sobre los cuales les voy a dejar algunas cosas para que lean al final de manera optativa por si les da curiosidad. Incluso, luego de los embeddings, surgieron recientemente los contextualized embeddings que además de considerar la relación semántica, consideran el orden puntual dentro de la oración.

Pero volviendo a las Bag Of Words, se puede hacer un pequeño "truco" para tener en cuenta, al menos parcialmente, algunas frases o expresiones con el orden en el que aparecen: el parámetro ngram_range calcula la frecuencia de ngramas. Resulta muy útil para descubrir frases o expresiones comunes (además de las palabras comunes). Además, en combinación con el parámetro "analyzer" se pueden usar como ngramas de palabras o de caracteres.

#### Ejercicio 3

Explorar los hiperparámetros de CountVectorizer. Ir modificando los valores de min_df, max_df y ngram-range. Observar cómo cambia el tamaño del vector.

NOTA: Como el tamaño del vector (es decir, el vocabulario) debe ser igual para el entrenamiento como para el test, tenemos que vectorizar al mismo tiempo el dataset de train y de test

In [76]:
text_train_abortion = abortion_train["Tweet"]
text_test_abortion = abortion_test["Tweet"]

vectorizer = CountVectorizer(
    binary=True, min_df=0.0075, max_df=0.75, ngram_range=(1, 5),
    #stop_words=stopwords.words('spanish')
)

X_abortion = vectorizer.fit_transform([*text_train, *text_test])

VEC_train_abortion = X_abortion[:len(text_train)]
VEC_test_abortion = X_abortion[len(text_train):]

Hacer esto mismo para los otros tres datasets (Cambio climático, feminismo y el del aborto balanceado)

In [ ]:
VEC_train_climate = #TODO
VEC_test_climate = #TODO

VEC_train_feminism = #TODO
VEC_test_feminism = #TODO

VEC_train_abortion_balanced = #TODO
VEC_test_abortion_balanced = #TODO

### EXTRA: Word Embeddings

Los embeddings de palabras son algo bastante nuevo en el campo del Procesamiento Del Lenguaje Natural (ultimos 10 años) pero fueron algo totalmente revolucionario que cambio absolutamente la disciplina. Desde que aparecieron las primeras versiones de embeddings (word2vec, glove, varias otras) surgieron muchas versiones distintas hechas con diversos algoritmos y técnicas. Pero todos tienen algo en común: tratan de captar la semántica de una palabra representandola con un vector (un número) que se calcula en base a los valores de las palabras que aparecen en el contexto de esa palabra. O sea, para cada palabra se calcula de manera iterativa un valor sobre la base de qué palabras aparecen antes y después en miles y miles de textos que se usan para entrenar los embeddings. Esos valores luego se exportan y se usan como representación de las palabras.

No es del alcance de este trabajo práctico meterse en este tema, que correspondería más a un curso introductorio de Procesamiento del Lenguaje Natural y ya no a Machine Learning, pero me pareció interesante comentarselos como una alternativa (muy muy) frecuente frente al problema de decidir como representar texto con números.

Si les interesa y quieren leer/investigar más al respecto, aca hay una clase del curso de PLN de Standford:
https://www.youtube.com/watch?v=ERibwqs9p38&list=PL3FW7Lu3i5Jsnh1rnUwq_TcylNr7EkRe6&index=2

La clase está buena aunque tiene bastante matemática. Es más que nada para que se entienda el concepto igualmente

# Clasificadores

A continuación van a realizar experimentos con tres clasificadores básicos. Para cada uno van a tener que probar una serie de hiperparámetros. Les incluyo la documentación para que puedan leer qué es cada hiperparámetro que están probando. Luego de cada corrida, evaluan el clasificador con cuatro métricas: Accuracy Score, F1 micro, F1 macro y el promedio del F1 de la clase Favor con el F1 de la clase Against. La idea es que vayan cambiando los valores de un hiperparámetro dejando fijos el resto y vean cómo ese cambio impacta en las métricas. Finalmente, para cada clasificador escriban un pequeño informe planteando cuan sensible es cada parámetro respecto de cada métrica, por qué piensan que es así de sensible y cuales son los mejores valores que encontraron. Finalmente, elijan el clasificador que les parezca más adecuado para esta tarea y justifiquen su elección. Para ese clasificador que hayan elegido van a probar luego, una busqueda más exhaustiva de hiperparámetros usando Grid Search. Este procedimiento deben hacerlo **al menos** para **dos de los cuatro** datasets con los que venimos trabajando (aborto, aborto balanceado, cambio climático y feminismo)

### Perceptron

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html

In [83]:
y_train = abortion_train["Stance"]
y_test = abortion_test["Stance"]


# En principio, pueden utilizar el módulo que sigue, con los parámetros por defecto y los que definan a continuación:
penalty =
alpha = 
max_iter =
tol =

model = Perceptron(penalty = penalty, alpha = alpha, fit_intercept=True, max_iter = max_iter, tol = tol, shuffle=True, random_state=0, class_weight=None, warm_start=False)
model.fit(VEC_train_abortion, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=1000, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty='l1', random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [90]:
y_pred_train =  model.predict(VEC_train_abortion)
accuracy_train = accuracy_score(y_train, y_pred_train)
f1_train_micro = f1_score(y_train, y_pred_train, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_train_macro = f1_score(y_train, y_pred_train, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_train = f1_score(y_train, y_pred_train, average=None, labels=["NONE", "AGAINST", "FAVOR"])
f1_train_average = #TODO

print("Accuracy para conjunto de entrenamiento: %.2f" % accuracy_train)
print("F1 micro para conjunto de entrenamiento: %.2f" % f1_train_micro)
print("F1 macro para conjunto de entrenamiento: %.2f" % f1_train_macro)
print("F1 average para conjunto de entrenamiento: %.2f" % f1_train_average)

y_pred_test =  model.predict(VEC_test_abortion)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
f1_test_average = #TODO

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

print("Exactitud del algoritmo para conjunto de test: %.2f" % accuracy_test)

SyntaxError: invalid syntax (<ipython-input-90-2d7fb9e7cdf1>, line 6)

### K-NN

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
n_neighbors =  # TODO: Cantidad de vecinos a tener en cuenta
metric =  # TODO: Medida de distancia. Algunas opciones: cosine, euclidean, manhattan.

model = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric)
model.fit(VEC_train_abortion, y_train)

In [ ]:
y_pred_train =  model.predict(VEC_train_abortion)
accuracy_train = accuracy_score(y_train, y_pred_train)
f1_train_micro = f1_score(y_train, y_pred_train, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_train_macro = f1_score(y_train, y_pred_train, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_train = f1_score(y_train, y_pred_train, average=None, labels=["NONE", "AGAINST", "FAVOR"])
f1_train_average = #TODO

print("Accuracy para conjunto de entrenamiento: %.2f" % accuracy_train)
print("F1 micro para conjunto de entrenamiento: %.2f" % f1_train_micro)
print("F1 macro para conjunto de entrenamiento: %.2f" % f1_train_macro)
print("F1 average para conjunto de entrenamiento: %.2f" % f1_train_average)

y_pred_test =  model.predict(VEC_test_abortion)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
f1_test_average = #TODO

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

print("Exactitud del algoritmo para conjunto de test: %.2f" % accuracy_test)

### Logistic Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
penalty =   # TODO: Tipo de regularización: l1 (valor absoluto), l2 (cuadrados).
alpha =   # TODO: Parámetro de regularización. También denominado como parámetro `lambda`. Debe ser mayor que 0.
max_iter =   # TODO: Cantidad máxima de iteraciones del algoritmo.
tol =   # TODO: Precisión del algoritmo (error mínimo entre una iteración y la siguiente).

model = LogisticRegression(penalty=penalty, C=1./alpha, max_iter=max_iter, tol=tol)
model.fit(VEC_train_abortion, y_train)

In [ ]:
y_pred_train =  model.predict(VEC_train_abortion)
accuracy_train = accuracy_score(y_train, y_pred_train)
f1_train_micro = f1_score(y_train, y_pred_train, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_train_macro = f1_score(y_train, y_pred_train, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_train = f1_score(y_train, y_pred_train, average=None, labels=["NONE", "AGAINST", "FAVOR"])
f1_train_average = #TODO

print("Accuracy para conjunto de entrenamiento: %.2f" % accuracy_train)
print("F1 micro para conjunto de entrenamiento: %.2f" % f1_train_micro)
print("F1 macro para conjunto de entrenamiento: %.2f" % f1_train_macro)
print("F1 average para conjunto de entrenamiento: %.2f" % f1_train_average)

y_pred_test =  model.predict(VEC_test_abortion)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
f1_test_average = #TODO

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

print("Exactitud del algoritmo para conjunto de test: %.2f" % accuracy_test)

### Grid Search

In [114]:

# Para la búsqueda de los mejores parámetros, por ejemplo de logistic regression, pueden usar:

exploring_params = {
        'C': [0.5, 1, 2, 5, 10, 20, 100, 200], # Inversa del coeficiente de regularización
        'max_iter': [1000, 5000, 10000],  # Cantidad de iteraciones
        'tol': [0.005, 0.002, 0.001, 0.0001]  # Precisión del algoritmo
    }

m = LogisticRegression()
n_cross_val =  2 # Seleccionar folds
scoring = "f1_micro"
model = GridSearchCV(m, exploring_params, cv=n_cross_val, scoring=scoring)
#    model.fit(X_train, y_train)
    
model.fit(VEC_train_abortion, y_train)

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")
print()
print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
print()
print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
y_true, y_pred = y_test, model.predict(VEC_test_abortion)
print(classification_report(y_true, y_pred), end="\n\n")

print("================================================", end="\n\n")

/home/damifur/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/damifur/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Mejor conjunto de parámetros:
{'C': 0.5, 'max_iter': 1000, 'tol': 0.005}


Puntajes de la grilla:


Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):

              precision    recall  f1-score   support

     AGAINST       0.82      0.72      0.77       189
       FAVOR       0.53      0.54      0.54        46
        NONE       0.35      0.51      0.41        45

   micro avg       0.66      0.66      0.66       280
   macro avg       0.57      0.59      0.57       280
weighted avg       0.70      0.66      0.67       280



